## 门控循环单元（GRU）
当时间步数较大或者时间步较小时，循环神经网络的梯度较容易出现衰减或爆炸。虽然裁剪梯度可以应对梯度爆炸，但无法解决梯度衰减的问题。通常由于这个原因，循环神经网络在实际中较难捕捉时间序列中时间步距离较大的依赖关系。

门控循环神经网络（gated recurrent neural network）的提出，正是为了更好地捕捉时间序列中时间步距离较大的依赖关系。它通过可以学习的门来控制信息的流动。其中，门控循环单元（gated recurrent unit，GRU）是一种常用的门控循环神经网络 。另一种常用的门控循环神经网络LSTM则将在下一节中介绍。

它引入了重置门（reset gate）和更新门（update gate）的概念，从而修改了循环神经网络中隐藏状态的计算方式。


<img src='gru.jpg' width='80%'>

### 重置门和更新门
门控循环单元中的重置门和更新门的输入均为当前时间步输入$X_t$与上一时间步隐藏状态$H_{t-1}$ ，输出由激活函数为sigmoid函数的全连接层计算得到。sigmoid函数可以将元素的值变换到0和1之间。因此，重置门$R_t$和更新门$Z_t$中每个元素的值域都是[0,1] .

### 候选隐藏状态
门控循环单元计算候选隐藏状态来辅助稍后的隐藏状态计算。将当前时间步重置门的输出与上一时间步隐藏状态做按元素乘法（符号为⊙）。如果重置门中元素值接近0，那么意味着重置对应隐藏状态元素为0，即丢弃上一时间步的隐藏状态。如果元素值接近1，那么表示保留上一时间步的隐藏状态。然后，将按元素乘法的结果与当前时间步的输入连结，再通过含激活函数tanh的全连接层计算出候选隐藏状态，其所有元素的值域为[−1,1]。从公式可以看出，重置门控制了上一时间步的隐藏状态如何流入当前时间步的候选隐藏状态。而上一时间步的隐藏状态可能包含了时间序列截至上一时间步的全部历史信息。因此，<font color='red'>重置门可以用来丢弃与预测无关的历史信息。</font>

### 隐藏状态
时间步t的隐藏状态$H_t$的计算使用当前时间步的更新门$Z_t$来对上一时间步的隐藏状态$H_{t-1}$和当前时间步的候选隐藏状态$ \tilde{H_t}$做组合. 更新门可以控制隐藏状态应该如何被包含当前时间步信息的候选隐藏状态所更新，如图6.6所示。假设更新门在时间步t′到t（t′<t）之间一直近似1。那么，在时间步t′到t之间的输入信息几乎没有流入时间步t的隐藏状态$H_t$。实际上，这可以看作是较早时刻的隐藏状态$H_{t′−1}$一直通过时间保存并传递至当前时间步t。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。
<p><font color='red'>
重置门有助于捕捉时间序列里短期的依赖关系；<br>
更新门有助于捕捉时间序列里长期的依赖关系。
</font></p>




## 长短期记忆LSTM
LSTM 中引入了3个门，即输入门（input gate）、遗忘门（forget gate）和输出门（output gate），以及与隐藏状态形状相同的记忆细胞（某些文献把记忆细胞当成一种特殊的隐藏状态），从而记录额外的信息

<img src='lstm.jpg' width='80%'>


###  输入门、输出门和遗忘门
与门控循环单元中的重置门和更新门一样，如图6.7所示，长短期记忆的门的输入均为当前时间步输入$X_t$与上一时间步隐藏状态$H_{t-1}$,输出由激活函数为sigmoid函数的全连接层计算得到。如此一来，这3个门元素的值域均为[0,1]。

### 候选记忆细胞
长短期记忆需要计算候选记忆细胞$\tilde{C_t}$。它的计算与上面介绍的3个门类似，但使用了值域在[−1,1]的tanh函数作为激活函数.

### 记忆细胞
通过元素值域在[0,1]的输入门、遗忘门和输出门来控制隐藏状态中信息的流动，这一般也是通过使用按元素乘法（符号为⊙）来实现的。当前时间步记忆细胞$C_t$的计算组合了上一时间步记忆细胞和当前时间步候选记忆细胞的信息，并通过遗忘门和输入门来控制信息的流动：

遗忘门控制上一时间步的记忆细胞$C_{t−1}$ 中的信息是否传递到当前时间步，而输入门则控制当前时间步的输入$X_t$ 通过候选记忆细胞 $ \tilde{C_t} $ 如何流入当前时间步的记忆细胞。如果遗忘门一直近似1且输入门一直近似0，过去的记忆细胞将一直通过时间保存并传递至当前时间步。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。
 
### 隐藏状态
通过输出门来控制从记忆细胞到隐藏状态$H_t$ 的信息的流动.这里的tanh函数确保隐藏状态元素值在-1到1之间。需要注意的是，当输出门近似1时，记忆细胞信息将传递到隐藏状态供输出层使用；当输出门近似0时，记忆细胞信息只自己保留。


## 深度神经网络 

在深度学习应用里，我们通常会用到含有多个隐藏层的循环神经网络，也称作深度循环神经网络。一个有L个隐藏层的深度循环神经网络，每个隐藏状态不断传递至当前层的下一时间步和当前时间步的下一层。
<img src='deep-rnn.svg' width='50%'>

$$
\boldsymbol{H}_t^{(1)} = \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(1)} + \boldsymbol{H}_{t-1}^{(1)} \boldsymbol{W}_{hh}^{(1)} + \boldsymbol{b}_h^{(1)})\\
\boldsymbol{H}_t^{(\ell)} = \phi(\boldsymbol{H}_t^{(\ell-1)} \boldsymbol{W}_{xh}^{(\ell)} + \boldsymbol{H}_{t-1}^{(\ell)} \boldsymbol{W}_{hh}^{(\ell)} + \boldsymbol{b}_h^{(\ell)})\\
\boldsymbol{O}_t = \boldsymbol{H}_t^{(L)} \boldsymbol{W}_{hq} + \boldsymbol{b}_q
$$

## 双向循环神经网络  bi-LSTM
双向循环神经网络通过增加从后往前传递信息的隐藏层来更灵活地处理这类信息。
<img src='bi-lstm.jpg' width=50%>